<a href="https://colab.research.google.com/github/naikasann/Keras_tutorial/blob/master/NurealNetwork_tutorial_2_Cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kerasを触ってみる(第2章 Cifar10をCNNで解いてみる)

次はCifar10と呼ばれるもう少し複雑な画像を判別することができるモデルを作成してみましょう。そのためにCNNと呼ばれる畳み込みニューラルネットワークを利用してみます。

ではやっていきましょう

---
## 全体の流れ

全体の流れはこのようになっています。これらを実行しながら、体験的に学んでいきます。

```
1. Cifar10のデータを見てみよう
2. モデルを定義する
3. 学習を行う
4. 学習結果を確認する
5. 実際に推論を行ってみる
6. おまけ
```
### 今回のゴール
``` 
Cifar10の10種類カテゴリを分類して、実際に分類をかけてみることができる
```

---

今回の参考記事

プログラムは

[cifar10とKerasを使ってCNN(Convolutional Neural Network)を実装してみる - Qiita](https://qiita.com/God_KonaBanana/items/10fa8bb58cdd1dbd2e59)

を利用させていただき、解説を自分なりのものに変えました。

---

## Kerasのライブラリをインポートする

まずはKerasなどののライブラリをインポートしていきます。

基本的には同じライブラリを用いて実行していきます。ただ今回はオプティマイザーやロス関数などのKeras内で用いられるライブラリなどを解説していきます。

1.   **オプティマイザー**
  学習していくうえでの最適化する手法の設定。
  RMSPropやAdam,SGDなどが有名です。
  前回のMNISTはRMSProp,今回はAdamを選択して学習を行っています。

  オプティマイザーはmodel.compileと呼ばれるモデルをコンパイルするメソッドを実行するときの引数として使用します。
  色々変えてみて試してみるといいと思います。

2.   **ロス関数(損失関数)**
  損失関数は学習の答えが近い値になるように重みパラメータを更新するのが目的です。
  学習のゴールを決めるような感じです。
  mean_squared_error(平均2乗誤差)やcategorical_crossentropy(カテゴリ交差エントロピー)、binary_crossentropy、kullback_leibler_divergenceなどが有名です。

  これもmodel.compileの引数として利用します。




In [ ]:
# 各種使用するライブラリをインポートします。
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

---

## Cifar10のデータを見てみよう。

深層学習のためには様々な技術が応用されていることが少しわかってもらえたかなと思います。次に実際のCifar10のデータを見てみたいと思います。Cifar10もKerasのライブラリですぐに導入することができます。

次に今回行う問題についてを見ていきましょう。
Cifar10は飛行機や犬などの10クラスを分類するような問題になっていています。学習データは5万枚、テストデータは1万枚とかなり大規模で実用的な分類をするような学習問題になっています。まずはどのような画像なのか見ていきましょう。

また深層学習に導入できるように整形までをここで行ってしまいます。

１０クラスは
```
1. airplane(飛行機)
2. automobile(自動車)
3. bird(鳥)
4. cat(猫)
5. deer(鹿)
6. dog(犬)
7. frog(カエル)
8. horse(馬)
9. ship(船)
10. truck(トラック)
```
の10種類を分類します。

In [ ]:
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

#5x5枚の画像を表示する
plt.figure(figsize=(10,10))
for i in range(25):
    rand_num=np.random.randint(0,50000)
    cifar_img=plt.subplot(5,5,i+1)
    plt.imshow(x_train[rand_num])
    #x軸の目盛りを消す
    plt.tick_params(labelbottom='off')
    #y軸の目盛りを消す
    plt.tick_params(labelleft='off')
    #正解ラベルを表示
    plt.title(y_train[rand_num])
plt.show()

#画像を0-1の範囲で正規化
x_train=x_train.astype('float32')/255.0
x_test=x_test.astype('float32')/255.0

#正解ラベルをOne-Hot表現に変換
y_train=to_categorical(y_train,10)
y_test=to_categorical(y_test,10)

このような画像が保存されています。画像が人間にはモザイクのように見えるのは32*32の画像サイズだからです。解像度を上げることでより詳細的に見ることができますが、コンピュータの処理能力の必要性が上がるため、少ない解像度で学習することが多いです。
また今回はMNISTと違ってRGBのカラー画像を用いて学習を行っていきます。

今回はデータ整形までを一気に行いました。

次はモデルの構造を定義していきます。

---

## モデルを定義する

モデルの構造を定義していきます。今回はCNNと呼ばれる畳み込みニューラルネットワークを用いて学習していくことにします。CNNの特徴は画像をあるサイズで区切りその画像で判別していくというものです。CNNは画像において優秀な成績を持っています。また近年では文章も区切って判別するようなCNN(FastText)などもあり、ニューラルネットワークの中でも一番といっていいほど有名なものになりました。

CNNはConv層と呼ばれる画像を区切る層と、Pooling層と呼ばれる画像の特徴を際立たせることが目的の層を並べることで実装されることが多いです。では実際にモデルを定義していきます。

In [ ]:
#モデルを構築
model=Sequential()

model.add(Conv2D(32,(3,3),padding='same',input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

# モデルアーキテクチャを画像に出力します
%matplotlib inline
plot_model(model, to_file='_.png', show_shapes=True, show_layer_names=True)
im = Image.open('_.png')
plt.figure(figsize=(10,10))
plt.axis('off')
_ = plt.imshow(np.array(im))

#モデルの表示
model.summary()

モデルは…

---

## 学習を行う

では学習を行ってみます。
今回はMNISTより多い回数学習を行いましょう

In [ ]:
# バッチサイズ
batch_size = 512

# 反復学習する回数
epochs = 20

# 学習の実行
history=model.fit(x_train,y_train,batch_size=128,epochs=20,verbose=1,validation_split=0.1)

学習にかなり時間が掛かったことが分かったと思います。
これは画像を畳み込み処理をしたことによって計算を行う処理の数や複雑性が上がっていることによって起きています。
今後も自分の用意した画像なども画像サイズなどによって学習する速度は変わってきます。ただ大きな画像を使うほど、より機械が詳細的にデータをみる機会が与えられることになるためより良い精度のモデルができる可能性が高まります。

---

## 学習結果を確認する

学習ができたら結果を見ていきましょう！まずは損失率や正解率を見ます。

In [ ]:
# 正答率と損失関数の値の推移をグラフ化します、
# 学習時の正答率などの情報は、history変数にすべて記録されています。

# 正答率の推移
plt.figure(figsize=(6,4))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.xticks(np.arange(0, 10, 2))
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# 損失関数の推移
plt.figure(figsize=(6,4))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.xticks(np.arange(0, 10, 2))
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

MNISTに比べて複雑な問題になっているはずですが、しっかりと学習が行えていることが分かってもらえると思います。

---

## 実際に推論を行ってみる。

推論が正しいのか確認します。


In [ ]:
# 表示するデータ数(頭から1～25まで対応)
display = 10

category = ("airplane",
            "automobile",
            "bird",
            "cat",
            "deer",
            "dog",
            "frog",
            "horse",
            "ship",
            "truck")

print("推論結果")
# 検証データに対する予測値の取得
y_predict = model.predict(x_test)
# 予測値を表示
print(y_predict[:display])
print("正解率")
# 正解データを表示
print(y_test[:display])

print("予測データ(整形後)")
# 予測した結果の中から一番確信度が高いデータを取り出す。(リストのインデックス)
predict_list = [np.argmax(prd) for prd in y_predict]
buff = predict_list[:display]
p_list = []
for c in buff:
  p_list.append(category[c])
print(p_list)
print("正解データ(整形後)")
# 正解データの生データを表示する。(生データの変数を上でなくしたので復元する)
ans_list = [np.argmax(ans) for ans in y_test]
buff = ans_list[:display]
p_list = []
for c in buff:
  p_list.append(category[c])
print(p_list)

# 推論を行ったMNISTデータの表示
fig = plt.figure(figsize=(9, 9))
fig.subplots_adjust(left=0, right=1, bottom=0, top=0.5, hspace=0.05, wspace=0.05)
for i in range(display):
    ax = fig.add_subplot(5, 5, i + 1, xticks=[], yticks=[])
    ax.imshow(x_test[i].reshape((32, 32, 3)))

このような形でCifar10のデータを分類することができました。
次はgoogle colabではなく、Pythonのスクリプトで自分が用意した画像データセットを学習してみてカメラで判別するようなものを作成してみましょう。

---

## おまけ

ネットで拾ってきた画像をCifar10のモデルで推論してみたくないですか？
そうすれば自分がその10種類の画像を分類することができるAIを作ることができるという実感がよりわくと思います。(記事の流れまんまですけどね…)
実際に推論するようなプログラムを置いておきますので試してみましょう。

まずは下のプログラムを実行して画像をアップロードしてください

In [ ]:
from google.colab import files

uploaded = files.upload()
!ls -l

次はその画像をロードしてモデルに推論していただきます。
下のプログラムで画像名を変換して実行してみてください

In [ ]:
imagename = "./OIP.jpg"

from keras.preprocessing.image import img_to_array, load_img

#画像読み込み
temp_img=load_img(imagename, target_size=(32,32))
#画像を配列に変換し0-1で正規化
temp_img_array=img_to_array(temp_img)
temp_img_array=temp_img_array.astype('float32')/255.0
temp_img_array=temp_img_array.reshape((1,32,32,3))

img_pred=model.predict_classes(temp_img_array)
print('\npredict_classes=', category[img_pred[0]])
